In [1]:
import h5py, os
import numpy as np

patterns=['P_2pt','N_2pt','T','D1ii','B_2pt','W_2pt','Z_2pt','M_correct_2pt','B','W','Z',\
          'NJN',\
            'pi0Insert','sigmaInsert','pi0Loop','sigmaLoop','insertLoop']

patternsMom=['N_2pt','T','D1ii','B_2pt','B','NJN','pi0Loop','sigmaLoop','insertLoop','pi0Insert','sigmaInsert']

In [2]:
# reduceTemplateSize

def dataQ(dataset):
    return not dataset.endswith('mvec')

for file in os.listdir('./'):
    if not file.endswith('.h5'):
        continue
    datasetDic={}
    with h5py.File(file,'r') as f:
        def visitor_func(name, node):
            if isinstance(node, h5py.Dataset):
                # print(name,node)
                if dataQ(name):
                    datasetDic[name]=0
                else:
                    datasetDic[name]=node[()]
        f.visititems(visitor_func)
    with h5py.File(file,'w') as f:
        for dataset in datasetDic:
            f.create_dataset(dataset,data=datasetDic[dataset])


In [3]:
# modify filenames

for file in os.listdir('./'):
    if not file.endswith('.h5'):
        continue
    if (not file.startswith('Diagram')) and (not file.startswith('threept')):
        continue
    for pat in patterns:
        if file.endswith(pat+'.h5'):
            print(file,pat)
            os.rename(file,pat+'.h5')

Diagram0000_sx00sy01sz18st039_NJN.h5 NJN


In [4]:
with h5py.File('templateMomList.h5','w') as fw:
    for pat in patternsMom:
        with h5py.File(pat+'.h5') as fr:
            dataset=None
            def visitor_func(name, node):
                if isinstance(node, h5py.Dataset):
                    if name.endswith('mvec'):
                        global dataset
                        dataset = name

            fr.visititems(visitor_func)
            fw.create_dataset(pat,data=fr[dataset])
            print("{0:20}".format(pat),"\t",fw[pat])

            if pat == 'N_2pt':
                t=[[0,0,0]+list(ele1)+list(-np.array(ele1))+list(ele2) for ele1 in fr[dataset] for ele2 in fr[dataset]]
                t.sort()
                t=np.array(t)
                fw.create_dataset('NpiJN',data=t)
                print("{0:20}".format('NpiJN'),"\t",fw['NpiJN'])

N_2pt                	 <HDF5 dataset "N_2pt": shape (27, 3), type "<i4">
NpiJN                	 <HDF5 dataset "NpiJN": shape (729, 12), type "<i8">
T                    	 <HDF5 dataset "T": shape (27, 6), type "<i4">
D1ii                 	 <HDF5 dataset "D1ii": shape (343, 9), type "<i4">
B_2pt                	 <HDF5 dataset "B_2pt": shape (343, 9), type "<i4">
B                    	 <HDF5 dataset "B": shape (27, 12), type "<i4">
NJN                  	 <HDF5 dataset "NJN": shape (27, 12), type "<i4">
pi0Loop              	 <HDF5 dataset "pi0Loop": shape (27, 3), type "<i4">
sigmaLoop            	 <HDF5 dataset "sigmaLoop": shape (27, 3), type "<i4">
insertLoop           	 <HDF5 dataset "insertLoop": shape (27, 3), type "<i4">
pi0Insert            	 <HDF5 dataset "pi0Insert": shape (27, 3), type "<i4">
sigmaInsert          	 <HDF5 dataset "sigmaInsert": shape (27, 3), type "<i4">
